# Automata inverse translator
## Goal
Create and train a Transformer to be able to build an automata from text.
## Inputs and Output
### Input
A text (string) describing all transitions in a wanted automata
### Output
A dot_string representation of an automata
## Steps to complete
### 1. Create a function to generate a random automata
I will now create a function that will generate an automata with the following alphabet : ["a", "b", ""]

In [45]:
import random, math
import numpy as np

def generate_automata():
    alphabet = ["a", "b", u"\u2205"]
    #nodes_quantity = random.randint(4,25)
    nodes_quantity = 5
    
    def generate_random_transitions(node_index):
        node_transitions = []
        while all(el == node_index for el in node_transitions):
            node_transitions = [random.randrange(nodes_quantity) for a in alphabet]
            
        return [(node_index, node_transitions[index], alphabet[index]) for index in range(len(node_transitions))]    
        
    automata = []
    for node in range(nodes_quantity):
        automata += generate_random_transitions(node)
        
    # Remove transitions
    transitions_to_remove = 0.3
    
    for i in range(math.ceil(len(automata) * transitions_to_remove)):
        np.random.shuffle(automata)
        automata = np.delete(automata, 0, 0)
    
    return automata

### 2. Generate dot_string from generated automata

In [46]:
def generate_dot_string(automata):
    dot_str = "digraph{"
    
    for transition in automata:
        dot_str += '%s->%s[label="%s"]' % tuple(transition)
    
    dot_str += "}"
    return dot_str

### 3. "Translate" transitions of the automata to english sentences

In [47]:
def translate_automata(automata):
    translation = ""
    
    def get_template():
        templates = [
            'When we are at state {source} and we get the value "{label}", then we go to state {target}.',
            'When we are at state {source} and we get the value "{label}", then we go to {target}.',
            'When we are at state {source} and the value is "{label}", then we go to state {target}.',
            'When we are at state {source} and the value is "{label}", then we go to {target}.',
        ]
        
        return templates[random.randrange(len(templates))]
    
    for transition in automata:
        translation += get_template().format(source = transition[0], target = transition[1], label = transition[2])
        translation += "\n"
    
    return translation

digraph {3->2 [label="b"] 1->1 [label="∅"] 3->3 [label="∅"] 2->2 [label="∅"] 0->4 [label="∅"] 4->0 [label="a"] 4->2 [label="∅"] 1->0 [label="a"] 2->0 [label="b"] 1->2 [label="b"] }
When we are at state 3 and we get the value "b", then we go to state 2.
When we are at state 1 and we get the value "∅", then we go to 1.
When we are at state 3 and we get the value "∅", then we go to 3.
When we are at state 2 and the value is "∅", then we go to state 2.
When we are at state 0 and the value is "∅", then we go to 4.
When we are at state 4 and we get the value "a", then we go to state 0.
When we are at state 4 and we get the value "∅", then we go to state 2.
When we are at state 1 and we get the value "a", then we go to state 0.
When we are at state 2 and we get the value "b", then we go to 0.
When we are at state 1 and we get the value "b", then we go to 2.



### 4. Train a transformer to build the dot_string from the sentences translated in #3
#### 4.1 Create a function to generate the dataset

In [ ]:
def generate_dataset():
    dataset_size = 1000
    dataset = []
    
    for i in range(dataset_size):
        automata = generate_automata()
        dataset.append([translate_automata(automata), generate_dot_string(automata)])
        
    return dataset

#### 4.2 Build the keras model